# Run logistic regression on a feature set

## Import data

In [ ]:
import pandas as pd
from os import path
from scipy.sparse import load_npz

dataDirectory = "./data/preprocessed-train-test"
featuresDirectory = "./data/features/tf-idf"

train, test, data = map(
    lambda filename: pd.read_csv(path.join(dataDirectory, filename)), 
    ["train.csv", "test.csv", "all.csv"])

trainFeatures, testFeatures, dataFeatures = map(
    lambda filename: load_npz(path.join(featuresDirectory, filename)),
    ["train.npz", "test.npz", "all.npz"])

## Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

def trainModel(features, labels):
    return LogisticRegression().fit(features, labels)

def getPredictions(model, features):
    return model.predict(features), model.predict_proba(features)

In [ ]:
model = trainModel(trainFeatures, train.toxic)

testPredictions, testProbPredictions = getPredictions(model, testFeatures)

## Evaluate predictions

In [ ]:
from evaluate_predictions import evaluatePredictions

In [ ]:
evaluatePredictions(test.toxic, testPredictions, testProbPredictions)

## Print metrics for train dataset

In [ ]:
trainPredictions, trainProbPredictions = getPredictions(model, trainFeatures)
evaluatePredictions(train.toxic, trainPredictions, trainProbPredictions)